<a href="https://colab.research.google.com/github/SwatiNeha/neural-networks-applied/blob/main/Tracking%20Random%20Walks%20with%20Sensor%20Networks%20and%20GNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tracking Random Walks with Sensor Networks and GNNs**

In [ ]:
! pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 566.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00


***1.*** ***Generating 256 Uniformly Distributed Random Sensors in Circular Area***

In [ ]:
import numpy as np
import torch
import plotly.graph_objects as go
import random

np.set_printoptions(threshold=np.inf)

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

def generate_random_sensors(num_sensors, radius):
    r = radius * np.sqrt(np.random.uniform(0, 1, num_sensors))
    theta = np.random.uniform(0, 2 * np.pi, num_sensors)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    sensors = np.column_stack((x, y))
    return sensors

num_sensors = 256
sensor_radius = 1000

# Generating sensor locations
sensor_locations = generate_random_sensors(num_sensors, sensor_radius)

for i, (x, y) in enumerate(sensor_locations):
    print(f"Sensor {i+1}: ({x:.2f}, {y:.2f})")



Sensor 1: (563.41, -238.97)
Sensor 2: (-324.34, -919.52)
Sensor 3: (-774.00, -364.58)
Sensor 4: (633.93, 443.61)
Sensor 5: (-296.28, -261.23)
Sensor 6: (394.19, -24.67)
Sensor 7: (153.52, 185.78)
Sensor 8: (-924.52, -106.95)
Sensor 9: (556.34, -540.00)
Sensor 10: (-48.78, -840.06)
Sensor 11: (-46.89, -135.60)
Sensor 12: (-289.68, -941.27)
Sensor 13: (-579.32, 704.86)
Sensor 14: (-124.64, 443.63)
Sensor 15: (155.38, -397.09)
Sensor 16: (157.94, -398.07)
Sensor 17: (370.12, -408.96)
Sensor 18: (619.41, -375.62)
Sensor 19: (-655.56, -46.80)
Sensor 20: (-539.63, -5.14)
Sensor 21: (233.73, -746.47)
Sensor 22: (-219.60, -302.11)
Sensor 23: (-160.66, -516.07)
Sensor 24: (171.76, -580.40)
Sensor 25: (520.36, -430.45)
Sensor 26: (-465.34, 754.08)
Sensor 27: (-317.12, 314.81)
Sensor 28: (595.67, 399.27)
Sensor 29: (-678.45, -363.49)
Sensor 30: (210.05, 48.26)
Sensor 31: (-761.31, 167.17)
Sensor 32: (-398.21, -109.33)
Sensor 33: (-58.05, 248.36)
Sensor 34: (-819.72, -526.25)
Sensor 35: (964.68, 1

***2.*** ***Calculating Sensors Adjacency Matrix - Communication Range: u= 100***

In [ ]:
def calculate_adjacency_matrix(sensors, communication_range):
    num_sensors = len(sensors)
    adjacency_matrix = np.zeros((num_sensors, num_sensors), dtype=int)
    for i in range(num_sensors):
        for j in range(i + 1, num_sensors):
            distance = np.sqrt((sensors[i, 0] - sensors[j, 0])**2 + (sensors[i, 1] - sensors[j, 1])**2)
            if distance <= communication_range:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    return adjacency_matrix

communication_range = 100
adjacency_matrix = calculate_adjacency_matrix(sensor_locations, communication_range)
print("Adjacency Matrix for Communication Range", communication_range, "meters:")
print(adjacency_matrix)

Adjacency Matrix for Communication Range 100 meters:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

***3.*** ***Creating Multiple Random Walks for Tiger with Nearest Sensor Detection and Addition of some False Positives***

In [ ]:
step_size = 50  # Step that a tiger takes in one walk
max_steps = 50000 # to limit the tiger walks
detection_radius = 50  # sensors are triggered when tiger passes within this meter
q = 0.05 # False positive probability
num_walks = 5 # Number of different random walks

# To add false positives to sensor readings
def add_false_positives(readings, q):
    noisy_readings = readings.copy()
    for i in range(len(readings)):
        if readings[i] == 0 and np.random.rand() < q:
            noisy_readings[i] = 1
    return noisy_readings

# Simulation of random walk from the center of a circular area
def random_walks_from_center(radius, step_size, max_steps, num_walks):
    walks_data = []
    for _ in range(num_walks):
        # Initialize the tiger's position at the center
        tiger_position = np.array([0.0, 0.0])
        walk_data = [tiger_position.copy()]

        for _ in range(max_steps):
            # Choose a random direction
            angle = np.random.uniform(0, 2 * np.pi)

            # Update the tiger's position
            tiger_position[0] += step_size * np.cos(angle)
            tiger_position[1] += step_size * np.sin(angle)

            # Check if the tiger is out of bounds
            if np.linalg.norm(tiger_position) > radius:
                break

            # Add the new position to the walk data
            walk_data.append(tiger_position.copy())

        walks_data.append(np.array(walk_data))  # Store each walk as an array

    return walks_data

# Detection of which sensors the tiger visited and adding false positives
def detect_sensors(walk_data, sensors, detection_radius, adjacency_matrix, q):
    sensor_visits = np.zeros(len(sensors))

    for tiger_position in walk_data:
        # Check nearest sensor location and mark it as visited if within detection radius
        distances_to_sensors = np.linalg.norm(sensors - tiger_position, axis=1)
        nearest_sensor_idx = np.argmin(distances_to_sensors)

        if distances_to_sensors[nearest_sensor_idx] <= detection_radius:
            sensor_visits[nearest_sensor_idx] = 1

            # Activating all sensors within communication range
            neighbors = np.where(adjacency_matrix[nearest_sensor_idx] == 1)[0]
            for neighbor in neighbors:
                sensor_visits[neighbor] = 1

    false_positives = add_false_positives(sensor_visits, q)
    return sensor_visits, false_positives


all_sensor_visits = np.zeros(len(sensor_locations), dtype=int)
all_false_positives = np.zeros(len(sensor_locations), dtype=int)

walks_data = random_walks_from_center(sensor_radius, step_size, max_steps, num_walks)

for walk_data in walks_data:
    sensor_visits, false_positives = detect_sensors(walk_data, sensor_locations, detection_radius, adjacency_matrix, q)
    all_sensor_visits = np.maximum(all_sensor_visits, sensor_visits)
    all_false_positives = np.maximum(all_false_positives, false_positives)

print("Total Sensor Visits (Aggregated over all walks):")
print(all_sensor_visits)
print("Total False Positives (Aggregated over all walks):")
print(all_false_positives)

print(f"Total number of steps for each walk: {[len(walk) - 1 for walk in walks_data]}")  # List of steps for each walk

Total Sensor Visits (Aggregated over all walks):
[1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0.]
Total False Positives (Aggregated over all walks):
[1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0

***4.*** ***Plotting the Random Walks with Activated Sensors and False Positives***

In [ ]:
fig = go.Figure()

# Plot non-visited sensors for the entire dataset
non_visited_sensors = sensor_locations[(all_sensor_visits == 0) & (all_false_positives == 0)]
fig.add_trace(go.Scatter(
    x=non_visited_sensors[:, 0],
    y=non_visited_sensors[:, 1],
    mode='markers',
    marker=dict(size=8, color='red', opacity=1),
    name='Non-Visited Sensors'
))

# Plot visited sensors for the entire dataset
visited_sensors = sensor_locations[all_sensor_visits > 0]
fig.add_trace(go.Scatter(
    x=visited_sensors[:, 0],
    y=visited_sensors[:, 1],
    mode='markers',
    marker=dict(size=12, color='darkblue', opacity=1, symbol = 'triangle-up'),
    name='Visited Sensors'
))

# Plot false positive sensors for the entire dataset
false_positive_sensors = sensor_locations[(all_false_positives > 0) & (all_sensor_visits == 0)]
fig.add_trace(go.Scatter(
    x=false_positive_sensors[:, 0],
    y=false_positive_sensors[:, 1],
    mode='markers',
    marker=dict(size=12, color='orange', opacity=1, symbol = 'x'),
    name='False Positive Sensors'
))

colors = ['orange', 'purple', 'cyan', 'magenta', 'pink']  # Color list for each walk
for i, walk_data in enumerate(walks_data):
    walk_x = walk_data[:, 0]
    walk_y = walk_data[:, 1]

    # Plot the random walk path
    fig.add_trace(go.Scatter(x=walk_x, y=walk_y, mode='lines', name=f'Tiger Random Walk {i+1}', opacity=0.5))

# Plot the circular boundary
circle_theta = np.linspace(0, 2 * np.pi, 500)
circle_x = sensor_radius * np.cos(circle_theta)
circle_y = sensor_radius * np.sin(circle_theta)
fig.add_trace(go.Scatter(
    x=circle_x,
    y=circle_y,
    mode='lines',
    line=dict(color='black', width=2, dash='dot'),
    name='Boundary'
))

# Update layout
fig.update_layout(
    title='Tiger Random Walks with Sensors, False Positives, and Circular Boundary',
    xaxis_title='X Coordinate (meters)',
    yaxis_title='Y Coordinate (meters)',
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1),
    width=1000,
    height=1000,
    showlegend=True
)

# Show the plot
fig.show()

***5.*** ***Creating Graph Data from all the Sensors for Training GNN***

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Create graph data for PyTorch Geometric
def create_graph_data(sensor_locations, adjacency_matrix, sensor_visits):
    x = torch.tensor(sensor_locations, dtype=torch.float)
    edge_index = torch.tensor(np.array(adjacency_matrix.nonzero()), dtype=torch.long)
    y = torch.tensor(sensor_visits, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, y=y)
    return data

graph_data_n = create_graph_data(sensor_locations, adjacency_matrix, all_false_positives)

***6.*** ***Splitting the Graph Data in Training and Validation Set***

In [ ]:
from sklearn.model_selection import train_test_split

def create_train_val_data(graph_data, test_size=0.2, random_state=42):
    # Create training and validation masks
    num_nodes = graph_data.num_nodes
    node_indices = np.arange(num_nodes)
    train_mask, val_mask = train_test_split(node_indices, test_size=test_size, random_state=random_state)

    # Create a mapping from original node indices to new indices in the training and validation sets
    train_idx_map = {idx: i for i, idx in enumerate(train_mask)}
    val_idx_map = {idx: i for i, idx in enumerate(val_mask)}

    # Filter edge_index to include only edges within the training set
    train_edges_mask = np.isin(graph_data.edge_index[0].numpy(), train_mask) & np.isin(graph_data.edge_index[1].numpy(), train_mask)
    train_edge_index = graph_data.edge_index[:, train_edges_mask]
    train_edge_index = torch.tensor([[train_idx_map[idx.item()] for idx in edge] for edge in train_edge_index.t()]).t()

    # Filter edge_index to include only edges within the validation set
    val_edges_mask = np.isin(graph_data.edge_index[0].numpy(), val_mask) & np.isin(graph_data.edge_index[1].numpy(), val_mask)
    val_edge_index = graph_data.edge_index[:, val_edges_mask]
    val_edge_index = torch.tensor([[val_idx_map[idx.item()] for idx in edge] for edge in val_edge_index.t()]).t()

    # Create train and val data objects
    train_data = Data(x=graph_data.x[train_mask], edge_index=train_edge_index, y=graph_data.y[train_mask])
    val_data = Data(x=graph_data.x[val_mask], edge_index=val_edge_index, y=graph_data.y[val_mask])

    return train_data, val_data

train_data, val_data = create_train_val_data(graph_data_n)

In [ ]:
train_data, val_data

(Data(x=[204, 2], edge_index=[2, 436], y=[204]),
 Data(x=[52, 2], edge_index=[2, 26], y=[52]))

***7.*** ***Defining The GAT : Graph Attention Network for Training***

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from sklearn.model_selection import train_test_split
import numpy as np

class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, output_dim, heads=4):
        super(GNNModel, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim1, heads=heads)
        self.fc = nn.Linear(hidden_dim1 * heads, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.fc(x)
        return torch.sigmoid(x)


***8.*** ***Training the Model without using Validation Set***

In [ ]:
input_dim = sensor_locations.shape[1]  # Number of features (2 for x and y coordinates)
hidden_dim1 = 20  # Hidden layer size
output_dim = 1  # Output dimension (1 for probability)
learning_rate = 0.001
epochs = 1000

model = GNNModel(input_dim, hidden_dim1, output_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Define the training procedure
def train_gnn_model(model, data, epochs):
    model.train()
    torch.manual_seed(42)
    loss_values = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index).squeeze()
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())
        if (epoch + 1) % 100 == 0:
          print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')
    return loss_values

loss_values = train_gnn_model(model, graph_data_n, epochs)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, len(loss_values) + 1)),
    y=loss_values,
    mode='lines',
    name='Training Loss'
))

fig.update_layout(
    title='Training Loss Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)

fig.show()

Epoch 100/1000, Loss: 0.5364348888397217
Epoch 200/1000, Loss: 0.4615846574306488
Epoch 300/1000, Loss: 0.43287479877471924
Epoch 400/1000, Loss: 0.4093567728996277
Epoch 500/1000, Loss: 0.38873958587646484
Epoch 600/1000, Loss: 0.37194281816482544
Epoch 700/1000, Loss: 0.35458841919898987
Epoch 800/1000, Loss: 0.33867695927619934
Epoch 900/1000, Loss: 0.3176462650299072
Epoch 1000/1000, Loss: 0.2970805764198303


***9.*** ***Testing the Model on False Positive + Truly Activated Sensors and comparing it with Truly Activated Sensors***

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        predictions = model(data.x, data.edge_index).squeeze()
        predicted_labels = (predictions > 0.5).float()
    return predicted_labels

def evaluate_performance(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    return accuracy, precision, recall, f1

true_labels = np.array(all_sensor_visits)  # Ground truth labels
predicted_labels = evaluate_model(model, graph_data_n).numpy()
accuracy, precision, recall, f1 = evaluate_performance(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9258
Precision: 0.9074
Recall: 0.9735
F1 Score: 0.9393


***10.*** ***HyperParamater Tuning of the Model with Validation Set to Prevent Overfitting***

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import ParameterGrid

# Define your parameters
input_dim = sensor_locations.shape[1]
output_dim = 1
seed = 42

# Define the hyperparameter search space
hyperparameters = {
    'heads': [1, 2, 4, 8],  # Number of attention heads in GNN layers
    'hidden_dim1': [8, 16, 18, 20],  # Size of hidden layers
    'learning_rate': [0.001, 0.005, 0.01],  # Learning rate for optimizer
    'epochs': [100, 200, 500, 1000],  # Number of training epochs
    'patience': [10, 20, 30]  # Patience for early stopping
}

# Generate all combinations of hyperparameters
param_grid = list(ParameterGrid(hyperparameters))

# Function to train the GNN model with specified hyperparameters
def train_gnn_model(input_dim, hidden_dim1, output_dim, train_data, val_data, epochs, patience, learning_rate, heads, seed):
    # Set the seed for reproducibility
    torch.manual_seed(seed)

    # Initialize model with hyperparameters
    model = GNNModel(input_dim, hidden_dim1, output_dim, heads=heads)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()
    best_val_loss = float('inf')
    patience_counter = 0
    loss_values = []
    val_loss_values = []

    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_data.x, train_data.edge_index).squeeze()
        loss = criterion(out, train_data.y)
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())

        # Validation
        model.eval()
        with torch.no_grad():
            val_out = model(val_data.x, val_data.edge_index).squeeze()
            val_loss = criterion(val_out, val_data.y)
            val_loss_values.append(val_loss.item())

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

        model.train()

    return model, loss_values, val_loss_values, best_val_loss

# Dictionary to store the results of each hyperparameter combination
tuning_results = []

# Perform grid search over hyperparameters
for params in param_grid:
    print(f"Training with parameters: {params}")

    # Train the model
    model, loss_values, val_loss_values, best_val_loss = train_gnn_model(
        input_dim=input_dim,
        hidden_dim1=params['hidden_dim1'],
        output_dim=output_dim,
        train_data=train_data,
        val_data=val_data,
        epochs=params['epochs'],
        patience=params['patience'],
        learning_rate=params['learning_rate'],
        heads=params['heads'],
        seed=seed
    )

    # Save the results
    tuning_results.append({
        'params': params,
        'model': model,
        'best_val_loss': best_val_loss,
        'loss_values': loss_values,
        'val_loss_values': val_loss_values
    })
    print(f"Validation Loss: {best_val_loss}")

# Select the best hyperparameters based on validation loss
best_result = min(tuning_results, key=lambda x: x['best_val_loss'])
best_params = best_result['params']
print(f"Best hyperparameters: {best_params}")

# Plot the training and validation loss for the best model
best_loss_values = best_result['loss_values']
best_val_loss_values = best_result['val_loss_values']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, len(best_loss_values) + 1)),
    y=best_loss_values,
    mode='lines',
    name='Training Loss'
))
fig.add_trace(go.Scatter(
    x=list(range(1, len(best_val_loss_values) + 1)),
    y=best_val_loss_values,
    mode='lines',
    name='Validation Loss'
))
fig.update_layout(
    title='Training and Validation Loss Over Epochs (Best Model)',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)
fig.show()

Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.001, 'patience': 10}
Validation Loss: 21.787479400634766
Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.001, 'patience': 20}
Validation Loss: 20.11107063293457
Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.001, 'patience': 30}
Validation Loss: 20.11107063293457
Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.005, 'patience': 10}
Validation Loss: 20.717144012451172
Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.005, 'patience': 20}
Validation Loss: 20.6778564453125
Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.005, 'patience': 30}
Validation Loss: 20.6778564453125
Training with parameters: {'epochs': 100, 'heads': 1, 'hidden_dim1': 8, 'learning_rate': 0.01, 'patience': 10}
Validat

***11.*** ***Training the Model with Best HyperParameter Values and Validation set***

In [ ]:
# Define your parameters
input_dim = sensor_locations.shape[1]
hidden_dim1 = 20
output_dim = 1
learning_rate = 0.001
epochs = 500
patience = 20  # Number of epochs to wait for improvement
seed = 42

model = GNNModel(input_dim, hidden_dim1, output_dim)
criterion = nn.BCELoss()

def train_gnn_model(input_dim, hidden_dim1, output_dim, train_data, val_data, epochs, patience, learning_rate, seed):
    # Set the seed for reproducibility
    torch.manual_seed(42)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    best_val_loss = float('inf')
    patience_counter = 0
    loss_values = []
    val_loss_values = []

    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_data.x, train_data.edge_index).squeeze()
        loss = criterion(out, train_data.y)
        loss.backward()
        optimizer.step()
        loss_values.append(loss.item())

        # Validation
        model.eval()
        with torch.no_grad():
            val_out = model(val_data.x, val_data.edge_index).squeeze()
            val_loss = criterion(val_out, val_data.y)
            val_loss_values.append(val_loss.item())

        if (epoch + 1) % 10 == 0:
          print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')


        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

        model.train()

    return loss_values, val_loss_values


# Train the model with early stopping
loss_values, val_loss_values = train_gnn_model(input_dim, hidden_dim1, output_dim, train_data, val_data, epochs, patience, learning_rate, seed)

# Plot the training and validation loss
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=list(range(1, len(loss_values) + 1)),
    y=loss_values,
    mode='lines',
    name='Training Loss'
))
fig.add_trace(go.Scatter(
    x=list(range(1, len(val_loss_values) + 1)),
    y=val_loss_values,
    mode='lines',
    name='Validation Loss'
))
fig.update_layout(
    title='Training and Validation Loss Over Epochs',
    xaxis_title='Epoch',
    yaxis_title='Loss'
)
fig.show()

Epoch 10/500, Loss: 16.832839965820312, Val Loss: 14.204146385192871
Epoch 20/500, Loss: 10.8614501953125, Val Loss: 9.246837615966797
Epoch 30/500, Loss: 7.17549467086792, Val Loss: 6.788067817687988
Epoch 40/500, Loss: 1.9368784427642822, Val Loss: 1.1366196870803833
Epoch 50/500, Loss: 1.0007569789886475, Val Loss: 0.9631770253181458
Epoch 60/500, Loss: 0.749415397644043, Val Loss: 0.7911690473556519
Epoch 70/500, Loss: 0.650652289390564, Val Loss: 0.7489979863166809
Epoch 80/500, Loss: 0.5692406296730042, Val Loss: 0.6791753172874451
Epoch 90/500, Loss: 0.5307296514511108, Val Loss: 0.6389312744140625
Epoch 100/500, Loss: 0.516914427280426, Val Loss: 0.6271164417266846
Epoch 110/500, Loss: 0.5075579881668091, Val Loss: 0.6213064789772034
Epoch 120/500, Loss: 0.4995412230491638, Val Loss: 0.6197108030319214
Epoch 130/500, Loss: 0.49114465713500977, Val Loss: 0.6151837110519409
Epoch 140/500, Loss: 0.4846968650817871, Val Loss: 0.6121534705162048
Epoch 150/500, Loss: 0.47827470302581

***12.*** ***Testing the above Model on the Dataset***

In [ ]:
true_labels = np.array(all_sensor_visits)  # Ground truth labels
predicted_labels = evaluate_model(model, graph_data_n).numpy()
accuracy, precision, recall, f1 = evaluate_performance(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.8047
Precision: 0.7790
Recall: 0.9338
F1 Score: 0.8494


In [ ]:
predicted_labels

array([1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 1., 1., 1.

***13.*** ***Plotting the Predicted Sensors with the Latest Model***

In [ ]:
# Define masks for various categories
true_positives = (true_labels == 1) & (predicted_labels == 1)
false_negatives = (true_labels == 1) & (predicted_labels == 0)
false_positives = (true_labels == 0) & (predicted_labels == 1)
true_negatives = (true_labels == 0) & (predicted_labels == 0)

# Get sensor locations for each category
true_positive_sensors = sensor_locations[true_positives]
false_negative_sensors = sensor_locations[false_negatives]
false_positive_sensors = sensor_locations[false_positives]
true_negative_sensors = sensor_locations[true_negatives]

# Plot setup
fig = go.Figure()

# Plot true positives
fig.add_trace(go.Scatter(
    x=true_positive_sensors[:, 0],
    y=true_positive_sensors[:, 1],
    mode='markers',
    marker=dict(size=12, color='darkblue', opacity=1, symbol = 'triangle-up'),
    name='True Positives'
))

# Plot true negatives
fig.add_trace(go.Scatter(
    x=true_negative_sensors[:, 0],
    y=true_negative_sensors[:, 1],
    mode='markers',
    marker=dict(size=8, color='red', opacity=1),
    name='True Negatives'
))

# Plot false negatives
fig.add_trace(go.Scatter(
    x=false_negative_sensors[:, 0],
    y=false_negative_sensors[:, 1],
    mode='markers',
    marker=dict(size=12, color='magenta', opacity=1, symbol = 'hexagram'),
    name='False Negatives'
))

# Plot false positives
fig.add_trace(go.Scatter(
    x=false_positive_sensors[:, 0],
    y=false_positive_sensors[:, 1],
    mode='markers',
    marker=dict(size=12, color='orange', opacity=1, symbol = 'x'),
    name='False Positives'
))

# Plot the random walk paths
colors = ['orange', 'purple', 'cyan', 'magenta', 'pink']  # Color list for each walk
for i, walk_data in enumerate(walks_data):
    walk_x = walk_data[:, 0]
    walk_y = walk_data[:, 1]

    fig.add_trace(go.Scatter(x=walk_x, y=walk_y, mode='lines', name=f'Tiger Random Walk {i+1}', opacity=0.5))

# Plot the circular boundary
circle_theta = np.linspace(0, 2 * np.pi, 500)
circle_x = sensor_radius * np.cos(circle_theta)
circle_y = sensor_radius * np.sin(circle_theta)
fig.add_trace(go.Scatter(
    x=circle_x,
    y=circle_y,
    mode='lines',
    line=dict(color='black', width=2, dash='dot'),
    name='Boundary'
))

# Update layout
fig.update_layout(
    title='Model Predictions with Sensors, Random Walks, and Circular Boundary',
    xaxis_title='X Coordinate (meters)',
    yaxis_title='Y Coordinate (meters)',
    xaxis=dict(scaleanchor="y", scaleratio=1),
    yaxis=dict(scaleanchor="x", scaleratio=1),
    width=1000,
    height=1000,
    showlegend=True
)

# Show the plot
fig.show()

***14.*** ***Evaluating Performance of this Model of 256 sensors with q, u and d values***

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        predictions = model(data.x, data.edge_index).squeeze()
        predicted_labels = (predictions > 0.5).float()
    return predicted_labels

def evaluate_performance(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    return accuracy, precision, recall, f1

# Function to evaluate the model on multiple walks
def evaluate_model_on_multiple_walks(model, sensor_locations, walks_data, q, u, d):
    adjacency_matrix = calculate_adjacency_matrix(sensor_locations, u)
    all_sensor_visits = np.zeros(len(sensor_locations), dtype=int)
    all_false_positives = np.zeros(len(sensor_locations), dtype=int)

    # Process each walk
    for walk_data in walks_data:
       sensor_visits, false_positives = detect_sensors(walk_data, sensor_locations, d, adjacency_matrix, q)
       all_sensor_visits = np.maximum(all_sensor_visits, sensor_visits)
       all_false_positives = np.maximum(all_false_positives, false_positives)

    data = create_graph_data(sensor_locations, adjacency_matrix, all_false_positives)

    true_labels = np.array(all_sensor_visits)  # Ground truth labels
    predicted_labels = evaluate_model(model, data).numpy()
    accuracy_1, precision_1, recall_1, f1_1 = evaluate_performance(true_labels, predicted_labels)

    return accuracy_1, precision_1, recall_1, f1_1

qs = [0.01, 0.05, 0.1]
us = [50, 100, 150]
ds = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

results = {}


for q in qs:
    walks_data1 = random_walks_from_center(sensor_radius, step_size, max_steps, num_walks=5)
    sensor_locations = generate_random_sensors(256, sensor_radius)
    for u in us:
        for d in ds:
            print(f"Evaluating q={q}, u={u}, d={d}...")
            avg_accuracy, avg_precision, avg_recall, avg_f1_score = evaluate_model_on_multiple_walks(
                model, sensor_locations, walks_data1, q, u, d
            )
            results[(q, u, d)] = {
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1_score
            }
            print(f'q={q}, u={u}, d={d} -> Accuracy: {avg_accuracy}, Precision: {avg_precision}, Recall: {avg_recall}, F1-score: {avg_f1_score}')


Evaluating q=0.01, u=50, d=10...
q=0.01, u=50, d=10 -> Accuracy: 0.4765625, Precision: 0.23717948717948717, Recall: 0.7115384615384616, F1-score: 0.3557692307692308
Evaluating q=0.01, u=50, d=20...
q=0.01, u=50, d=20 -> Accuracy: 0.59765625, Precision: 0.5128205128205128, Recall: 0.7476635514018691, F1-score: 0.6083650190114068
Evaluating q=0.01, u=50, d=30...
q=0.01, u=50, d=30 -> Accuracy: 0.63671875, Precision: 0.6089743589743589, Recall: 0.7480314960629921, F1-score: 0.6713780918727914
Evaluating q=0.01, u=50, d=40...
q=0.01, u=50, d=40 -> Accuracy: 0.62890625, Precision: 0.6410256410256411, Recall: 0.7194244604316546, F1-score: 0.6779661016949153
Evaluating q=0.01, u=50, d=50...
q=0.01, u=50, d=50 -> Accuracy: 0.6328125, Precision: 0.6602564102564102, Recall: 0.7152777777777778, F1-score: 0.6866666666666666
Evaluating q=0.01, u=50, d=60...
q=0.01, u=50, d=60 -> Accuracy: 0.6328125, Precision: 0.6730769230769231, Recall: 0.7094594594594594, F1-score: 0.6907894736842106
Evaluating q

***15.*** ***Plotting the Accuracy for different q and u wrt to d***

In [ ]:
from itertools import groupby

fig_accuracy = go.Figure()
# Sort and group the keys
sorted_keys = sorted(results.keys(), key=lambda x: (x[0], x[1]))
grouped = groupby(sorted_keys, key=lambda x: (x[0], x[1]))

for (q, u), group in grouped:
    d_values = [x[2] for x in group]
    accuracies = [results[(q, u, d)]['accuracy'] for d in d_values]
    fig_accuracy.add_trace(go.Scatter(
        x=d_values,
        y=accuracies,
        mode='lines+markers',
        name=f'q={q}, u={u}'
    ))

fig_accuracy.update_layout(
    title='Accuracy vs. d for Different q and u Values',
    xaxis_title='d',
    yaxis_title='Accuracy',
    legend_title='q and u'
)

fig_accuracy.show()

***16.*** ***Generating 512 Uniformly Distributed Sensors inside a Circle and Calculating the Adjacency Matrix***

In [ ]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

np.set_printoptions(threshold=np.inf)
num_sensors = 512
sensor_radius = 1000
sensor_locations = generate_random_sensors(num_sensors, sensor_radius)
adjacency_matrix = calculate_adjacency_matrix(sensor_locations, communication_range)

***17.*** ***With the same Random walks used for 256 model - Detecting the sensors that are activated out of 512 sensors nd adding False Positives as well***

In [ ]:
step_size = 50  # Step that a tiger takes in one walk
max_steps = 50000 # to limit the tiger walks
detection_radius = 50  # sensors are triggered when tiger passes within this meter
q = 0.05 # False positive probability
num_walks = 5 # Number of different random walks

all_sensor_visits = np.zeros(len(sensor_locations), dtype=int)
all_false_positives = np.zeros(len(sensor_locations), dtype=int)

for walk_data in walks_data:
    sensor_visits, false_positives = detect_sensors(walk_data, sensor_locations, detection_radius, adjacency_matrix, q)
    all_sensor_visits = np.maximum(all_sensor_visits, sensor_visits)
    all_false_positives = np.maximum(all_false_positives, false_positives)

print("Total Sensor Visits (Aggregated over all walks):")
print(all_sensor_visits)
print("Total False Positives (Aggregated over all walks):")
print(all_false_positives)

print(f"Total number of steps for each walk: {[len(walk) - 1 for walk in walks_data]}")  # List of steps for each walk

Total Sensor Visits (Aggregated over all walks):
[1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1.
 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1.
 1

***18.*** ***Creating Graph Data from the Sensor matrix***

In [ ]:
graph_data_n = create_graph_data(sensor_locations, adjacency_matrix, all_false_positives)

***19.*** ***Evaluating Perfornce of the Model with 512 sensors on q, u and d values***

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        predictions = model(data.x, data.edge_index).squeeze()
        predicted_labels = (predictions > 0.5).float()
    return predicted_labels

def evaluate_performance(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    return accuracy, precision, recall, f1

# Function to evaluate the model on multiple walks
def evaluate_model_on_multiple_walks(model, sensor_locations, walks_data, q, u, d):
    adjacency_matrix = calculate_adjacency_matrix(sensor_locations, u)
    all_sensor_visits = np.zeros(len(sensor_locations), dtype=int)
    all_false_positives = np.zeros(len(sensor_locations), dtype=int)

    # Process each walk
    for walk_data in walks_data:
       sensor_visits, false_positives = detect_sensors(walk_data, sensor_locations, d, adjacency_matrix, q)
       all_sensor_visits = np.maximum(all_sensor_visits, sensor_visits)
       all_false_positives = np.maximum(all_false_positives, false_positives)

    data = create_graph_data(sensor_locations, adjacency_matrix, all_false_positives)

    true_labels = np.array(all_sensor_visits)  # Ground truth labels
    predicted_labels = evaluate_model(model, data).numpy()
    accuracy_1, precision_1, recall_1, f1_1 = evaluate_performance(true_labels, predicted_labels)

    return accuracy_1, precision_1, recall_1, f1_1

qs = [0.01, 0.05, 0.1]
us = [50, 100, 150]
ds = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

results = {}


for q in qs:
    walks_data1 = random_walks_from_center(sensor_radius, step_size, max_steps, num_walks=5)
    sensor_locations = generate_random_sensors(512, sensor_radius)
    for u in us:
        for d in ds:
            print(f"Evaluating q={q}, u={u}, d={d}...")
            avg_accuracy, avg_precision, avg_recall, avg_f1_score = evaluate_model_on_multiple_walks(
                model, sensor_locations, walks_data1, q, u, d
            )
            results[(q, u, d)] = {
                'accuracy': avg_accuracy,
                'precision': avg_precision,
                'recall': avg_recall,
                'f1': avg_f1_score
            }
            print(f'q={q}, u={u}, d={d} -> Accuracy: {avg_accuracy}, Precision: {avg_precision}, Recall: {avg_recall}, F1-score: {avg_f1_score}')


Evaluating q=0.01, u=50, d=10...
q=0.01, u=50, d=10 -> Accuracy: 0.4921875, Precision: 0.29245283018867924, Recall: 0.7265625, F1-score: 0.4170403587443946
Evaluating q=0.01, u=50, d=20...
q=0.01, u=50, d=20 -> Accuracy: 0.576171875, Precision: 0.5157232704402516, Recall: 0.7224669603524229, F1-score: 0.601834862385321
Evaluating q=0.01, u=50, d=30...
q=0.01, u=50, d=30 -> Accuracy: 0.59765625, Precision: 0.5880503144654088, Recall: 0.7137404580152672, F1-score: 0.6448275862068966
Evaluating q=0.01, u=50, d=40...
q=0.01, u=50, d=40 -> Accuracy: 0.623046875, Precision: 0.6289308176100629, Recall: 0.7272727272727273, F1-score: 0.6745362563237773
Evaluating q=0.01, u=50, d=50...
q=0.01, u=50, d=50 -> Accuracy: 0.6171875, Precision: 0.6415094339622641, Recall: 0.7132867132867133, F1-score: 0.6754966887417219
Evaluating q=0.01, u=50, d=60...
q=0.01, u=50, d=60 -> Accuracy: 0.634765625, Precision: 0.6698113207547169, Recall: 0.7220338983050848, F1-score: 0.6949429037520392
Evaluating q=0.01,

***20.*** ***Plotting the Accuracy for different q and u wrt to d***

In [ ]:
from itertools import groupby

fig_accuracy = go.Figure()
# Sort and group the keys
sorted_keys = sorted(results.keys(), key=lambda x: (x[0], x[1]))
grouped = groupby(sorted_keys, key=lambda x: (x[0], x[1]))

for (q, u), group in grouped:
    d_values = [x[2] for x in group]
    accuracies = [results[(q, u, d)]['accuracy'] for d in d_values]
    fig_accuracy.add_trace(go.Scatter(
        x=d_values,
        y=accuracies,
        mode='lines+markers',
        name=f'q={q}, u={u}'
    ))

fig_accuracy.update_layout(
    title='Accuracy vs. d for Different q and u Values',
    xaxis_title='d',
    yaxis_title='Accuracy',
    legend_title='q and u'
)

fig_accuracy.show()

************************************************************************* ***END*** *************************************************************************************************